## 5.4 Factorization Methods

### 5.4.1 Cholesky Iteration

In [ ]:
import numpy as np
from scripts.lu import cholesky

**Implementation 5.17: Cholesky Iteration**

In [ ]:
def cholesky_iter(A, k):
    A = A.copy()
    n, m = A.shape
    for l in range(k):
        L = cholesky(A)
        A = L.transpose() @ L
        # for i in range(n):
        #     for j in range(n):
        #         a = 0
        #         for k in range(max(i, j), n):
        #             a += L[k, i] * L[k, j]
        #         A[i, j] = a
        # print(f'A_{l} =\n{A}\n')
    return A

*Additional code details*
- We use the `numpy` matrix product, as this is much faster than two nested python loops.
- To measure the computational time needed to execute a jupyter cell, we can use so called "cell magic". By writing `%%timit` into the first line of the cell, the content of that cell is repeated multiple times and the average computational time is reported. Try it in the next cell to see the efficiency gain between the `numpy` matrix product and the nested python loops.

#### Example 5.24 (Cholesky Iteration)
We consider the matrix
$$A = \begin{pmatrix}3&-1&0&1\\-1&3&1&1\\ 0&1&3&0\\1&1&0&3\end{pmatrix},$$
and compute several steps of the Cholesky iteration using $A_0:=A$.

In [ ]:
A = np.array([[3, -1, 0, 1],
              [-1, 3, 1, 1],
              [0, 1, 3, 0],
              [1, 1, 0, 3]], dtype=np.double)
A_chol = cholesky_iter(A, 50)

The eigenvalues are approximated by

In [ ]:
lam = np.flip(np.diag(A_chol))
print(lam)

This results in the following relative error

In [ ]:
lam_ex = np.linalg.eig(A)[0]
print(np.abs(lam - lam_ex) / lam_ex)

### 5.4.2 The QR Iteration

We implememnt the QR iteration using our implementation of the QR factorization using Givens rotations.

In [ ]:
from scripts.qr import qr_givens

In [ ]:
def qr_iter(A, k):
    A = A.copy()
    for l in range(k):
        QT = qr_givens(A)
        A[:, :] = A @ QT.transpose()
    return A

#### Example 5.28 (QR iteration) 

We compute twenty-five steps of the QR iteration.  

In [ ]:
A1 = qr_iter(A, 25)
lam_qr = np.flip(np.diag(A1))
print(lam_qr)

This results in the following relative errors

In [ ]:
print(np.abs(lam_qr - lam_ex) / lam_ex)

We see that only half the number of steps are necessary to achieve the same level of accuracy compared to the Cholesky iteration

#### Example 5.30 (QR iteration with shift)

We implement the QR iteration with shift and use the largest approximated eigenvalue as the shift.

In [ ]:
def qr_iter_shift(A, k):
    A = A.copy()
    Id = np.identity(A.shape[0], dtype=A.dtype)
    for l in range(k):
        mu = np.amax(np.diag(A))
        A[:,:] -= mu * Id
        QT = qr_givens(A)
        A[:, :] = A @ QT.transpose() + mu * Id
        # print(f'mu_0 = {mu}')
        # print(f'A_{l} =\n{A}\n')
    return A

Applying this to the same matrix in six steps only, we obtain

In [ ]:
A1 = qr_iter_shift(A, 6)
lam_qrs = np.diag(A1)
print(lam_qr)

This yields the relative errors

In [ ]:
print(np.abs(lam_qrs - lam_ex) / lam_ex)

i.e., after only six steps, we again have a maximum relative error of 0.3%.